In [1]:
# https://www.bindingdb.org/bind/chemsearch/marvin/SDFdownload.jsp?all_download=yes
# BindingDB_BindingDB_Inhibition_2021m6.tsv.zip

from rdkit import Chem
from rdkit.Chem import AllChem, PandasTools 
from rdkit.Chem.Draw import IPythonConsole  
import pandas as pd, sqlite3, math
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None) 

In [2]:
con = sqlite3.connect('/scratch/xg590/scratch/BindingDB.db') #  :memory:
cur = con.cursor()

In [3]:
df = pd.read_pickle('data/df_processed.pkl') 
df[['InChIKey','binder_smiles','unp_accessionid','unp_resnum','record_id']].to_sql('pdbcov',con,if_exists="replace",index=True)  

In [4]:
pd.read_sql('select * from bindingdb limit 1', con)

,react_id,inchikey,unprotid
0,143,ZYQKMYRXVHUATB-UHFFFAOYSA-N,P08684


In [5]:
pd.read_sql('select * from pdbcov limit 1', con)

,adduct_id,InChIKey,binder_smiles,unp_accessionid,unp_resnum,record_id
0,1C7Q_A_HIS_306_?_ND1_?_A_BE1_500_?_C1_?,MUORYJVNLAHVKL-UHFFFAOYSA-N,O=C(CBr)NCCOP(=O)(O)O,P13376,306,CBR000001


In [6]:
Mapping = pd.read_sql_query('''SELECT pdbcov.adduct_id, bindingdb.react_id, unp_accessionid, binder_smiles, record_id
                               FROM pdbcov JOIN bindingdb
                               ON     pdbcov.unp_accessionid = bindingdb.unprotid
                                  AND pdbcov.InChIKey        = bindingdb.inchikey''' , con) 
# executed in 39.3s, finished 06:37:50 2021-05-21

In [7]:
Mapping.drop_duplicates(['unp_accessionid','binder_smiles'])

,adduct_id,react_id,unp_accessionid,binder_smiles,record_id
0,3IKA_A_CYS_797_?_SG_?_A_0UN_1797_?_CAP_?,479094,P00533,C=CC(=O)Nc1cccc(Oc2nc(Nc3ccc(N4CCN(C)CC4)cc3OC)ncc2Cl)c1,CBR000181
38,1AHT_H_SER_195_?_OG_?_H_APA_401_?_C2_?,50089760,P00734,N=C(N)c1ccc(CC(=O)C(=O)O)cc1,CBR000221
39,3K83_A_SER_241_?_OG_?_A_F27_1_?_C13_?,50856616,P97612,O=C(O)c1cccc(-c2cnc(C(=O)CCc3ccc(-c4ccccc4)cc3)o2)n1,CBR000239
43,1YT7_A_CYS_25_?_SG_?_A_3FC_301_?_C31_?,50312286,P43235,CCCC[C@H](NC(=O)O[C@H](Cc1nnc(-c2ccc(C(F)(F)F)cc2)o1)C(C)(C)C)C(=O)C(=O)NN1CCOC1=O,CBR000247
45,7DUR_R_CYS_347_?_SG_?_R_HNO_701_?_C12_?,50145013,P43220,CC(C)(C)Nc1nc2cc(Cl)c(Cl)cc2nc1S(C)(=O)=O,CBR000254
...,...,...,...,...,...
2705,2OZ2_A_CYS_25_?_SG_?_A_D1R_301_?_C27_?,51233499,P25779,CN1CCN(C(=O)N[C@@H](Cc2ccccc2)C(=O)N[C@H](C=CS(=O)(=O)c2ccccc2)CCc2ccccc2)CC1,CBR007489
2711,1LHD_H_SER_195_?_OG_?_H_DI2_400_?_B1_?,50479752,P00734,CC(=O)N[C@H](Cc1ccccc1)C(=O)N1CCC[C@H]1C(=O)N[C@@H](CCCCN)B(O)O,CBR007515
2712,1LHC_H_SER_195_?_OG_?_H_DP7_400_?_B2_?,50479734,P00734,CC(=O)N[C@H](Cc1ccccc1)C(=O)N1CCC[C@H]1C(=O)N[C@@H](CCCN=C(N)N)B(O)O,CBR007518
2714,6TFP_A_CYS_481_?_SG_?_A_N6Z_702_?_C37_?,451867,Q06187,C=CC(=O)N(C)CCOc1c(N)ncnc1-c1cc(F)cc(NC(=O)c2ccc(C3CC3)cc2F)c1C,CBR007536


In [8]:
mapping = Mapping.sort_values('record_id')\
                 .drop_duplicates(['record_id'])[['record_id','react_id']]\
                 .set_index('record_id')\
                 .to_dict('dict')['react_id']
df['BindingDB_react_id']=df.apply(lambda r: str(mapping[r.record_id]) if r.record_id in mapping else math.nan,axis=1)
df.to_pickle('data/df_processed.pkl') 